# 8. Flask - Modular Architecture II: Blueprints
We have built a robust core for our control system using the `application factory`. Now it's time to extend this core with specialized modules that will handle specific parts of our operation, such as crew management or the data API. For this purpose, we will use `Blueprints` – an elegant way to divide our application into logical, reusable, and independently manageable components.

## 8.1. Blueprints & Their Use
- An application using **Blueprints** has:
  - A common `global environment`.
  - Several different `nested environments` (Blueprints - BP).

`Blueprints`:
- We can have `any number` of BPs in the application.
- We can add and separate BPs in the application (in the Application Factory) as needed.
- Each BP will have "its own things" - routes, files, forms, permissions, settings, style, and appearance, or whatever else it needs - and at the same time, it can also use `global elements` (base templates and HTML elements, files, layout ..).
- If we set **`template_folder=`** for HTML files when defining a BP - Flask will look for templates `first in the specified BP folder` - and if it doesn't find the file, it will use the **global `templates/` folder**.

- We will use Blueprints for:
  1. An application composed of different elements / components - user, admin, API ..
  2. If we want to reuse Blueprint code between multiple projects - e.g., authentication, API.
  3. Easily adding logically grouped routes to the application.
- An application with BPs can be easily developed in a team - each developer works on their own BP, and there are no collisions or overlapping responsibilities.

### 8.1.1. Blueprint Namespace & `url_for()`

When you register a Blueprint, Flask takes all of its routes and encloses them within a **namespace** that shares the same name as the Blueprint (e.g., `main`).

Think of it like files and folders. Originally, your `homepage` function was like a file in the root directory. Now, it's the `homepage` file **inside the `main` folder**.

For this reason, you can no longer reference the function by its name alone. You must now also tell `url_for()` the "folder" (the Blueprint's name) where it is located.

**The correct syntax takes the form `url_for('blueprint_name.function_name')`:**

In [ ]:
<!-- "url_for" link in HTML file -->

<!-- without Blueprint / global -->
<a href="{{ url_for('contact') }}">Kontakt</a>

<!-- with Blueprint 'main' -->
<a href="{{ url_for('main.contact') }}">Kontakt</a>

In [ ]:
# "url_for" in python file

# without Blueprint / global
return redirect(url_for('main_page'))

# with Blueprint 'general'
return redirect(url_for('general.main_page'))

### "Within" Blueprint Syntax - a shortcut

- If we are referring with `url_for()` between routes that are part of the **`same Blueprint`**, we can use a shortened form:
  - `url_for('.function_name')`
- The dot `.` replaces the name of the **current** Blueprint.

## Summary

- All links in the `url_for('something')` format, `that were moved to a BP`, must be modified for correct redirection and route distinction between different Blueprints:
  1.  `url_for('function_name')` → for **global routes**.
  2.  `url_for('blueprint.function')` → **when using a BP**.
  3.  `url_for('.function')` → simplified syntax **inside the same BP**.

It's a small **price** of modularity :)

## 8.2. ADVANCED Structure - Medium and Large Applications
Our system has evolved from a prototype to a functional solution that is robust enough on its own, but all commands are processed by one department. For more complex missions, we will start scaling - we will divide our application into specialized modules (Blueprints) according to their focus and the type of operations performed.

- **Blueprints** are suitable for `multi-purpose applications`.
- Where we need a highly modular environment.
- Additional added **environments** will be `marked` with a specific **Blueprint** - e.g., public, private, admin, etc.

**Application structure**:
- my_project/
  - wsgi.py
  - app/
    - _ _ init _ _.py
    - database.py
    - models/ 
      - _ _ init _ _.py 
      - drone_operator.py
    - templates/
    - static/
    - config/
    - **main/** Blueprint for global / common routes
      - **_ _ init _ _.py** - an existing `__init__.py` file will "mark" the folder as a package
      - **routes.py** - routes for the main / common interface
      - **form.py** - common forms
    - **API/** Blueprint for API routes
      - **_ _ init _ _.py** - DITTO
      - **routes.py** - API routes
      - **form.py** - API forms

- The file with our **SQLite DB** is in the default **instance** folder outside our application.

### 8.2.1. Blueprint - MAIN

In [ ]:
# app/main/__init__.py

from flask import Blueprint

# We create a Blueprint named 'main'
# 1st argument ('main') is the internal name that Flask will use for the BP - e.g., for url_for()
# 2nd argument (__name__) helps Flask find the correct folders (e.g., for templates)
main_bp = Blueprint('main', __name__)


# we import the file with routes = so the routes are registered on our blueprint
from . import routes # import is done AFTER creating the Blueprint (main_bp)

In [ ]:
# app/main/routes.py

from flask import render_template
from . import main_bp # we import the blueprint object from the same package (from __init__.py)

# Routes are now registered on the 'main_bp' blueprint, not on 'app'
@main_bp.route("/earth") 
def mission_earth():
    return """
            <h1>Mission</h1>
            <p>Welcome to Earth !</p>
        """
    
@main_bp.route("/mars")
def mission_mars():
    return """
            <h1>Mission</h1>
            <p>Welcome to Mars !</p>
        """
    
@main_bp.route("/venus")
def mission_venus():
    return """
            <h1>Mission</h1>
            <p>Welcome to Venus !</p>
        """

# ... and so on for all other routes that belong to the main part ...

### 8.2.2. Blueprint - API

In [ ]:
# app/api/__init__.py

from flask import Blueprint

# We create a Blueprint named 'api'
# 1st argument ('api') is the internal name that Flask will use for the BP - e.g., for url_for()
# 2nd argument (__name__) helps Flask find the correct folders (e.g., for templates)
api_bp = Blueprint('api', __name__)

# we import the file with routes = so the routes are registered on our blueprint
from . import routes # import AFTER Blueprint

In [ ]:
# app/api/routes.py

from flask import Flask, jsonify, make_response
from ..static.data.test_data import probe_data
from . import api_bp # we import the blueprint object from the same package (from __init__.py)


# we import the blueprint object from the same package (from __init__.py)
@api_bp.route("/probe_velocities", methods=["GET"])
def probe_velocities():
    # data is usually obtained from a DB
    probe_velocities = {"Velocities": [probe["velocity"] for probe in probe_data]}
    return jsonify(probe_velocities), 200


@api_bp.route("/probe_info/<int:probe_id>", methods=["GET"]) 
def probe_detail(probe_id):
    # data is usually obtained from a DB
    if not 1 <= probe_id <= len(probe_data): 
        return jsonify({"error": "Probe ID is out of range. Not found."}), 404
    
    probe = probe_data[probe_id - 1]
    return jsonify(ID_Probe=probe_id, Details=probe), 200
        

@api_bp.route('/probe_mothership', methods=['GET'])
def probe_mothership():
    # body 
    response = make_response(jsonify(
        {"message": "No data to fetch, mothership empty, systems still running"}
        ), 202)
    
    # headers 
    response.headers["Cache-Control"] = "no-store" 
    response.headers["Content-Language"] = "en" 

    # cookies
    response.set_cookie(
        key="session_mothership",
        value="50",
        max_age=3600,              # 1 hour
        httponly=True,
        secure=True,
        samesite="Lax"
    )
    return response

### 8.2.3. Registering Blueprints in the Application Factory

In [ ]:
# app/__init__.py

from flask import Flask
from .config.app_config_class import DBConfig
from .database import db
from .models import Drone_Operator
from .main import main_bp # import the blueprint object from 'main'
from .api import api_bp # import the blueprint object from 'api'


def create_app(): # The user has been using app_factory, let's be consistent
    app = Flask(__name__)

    # 1. CONFIGURATION
    app.config.from_object(DBConfig)

    # 2. EXTENSION INITIALIZATION - if we have extensions for the "app" object - we add them after configuration
    db.init_app(app) # extending "Flask app" with "SQLAlchemy db"

    # 3. BLUEPRINT REGISTRATION - we add BPs/routes after plugins
    # The optional 'url_prefix' argument adds a prefix to ALL ROUTES of the given Blueprint - instead of /my_route = /bp_name/my_route
    app.register_blueprint(main_bp) # 'app' learns about all routes, templates, forms, etc. defined in 'main_bp'
    app.register_blueprint(api_bp, url_prefix='/api') # DITTO for 'api_bp'


    # ------------
    # if we need to perform actions "in context", we do them last
    
    # 4. APPLICATION CONTEXT - actions that need to know the configuration and location 
    with app.app_context(): # I use "app_context()
        db.create_all()  # creates all defined tables (models) - IF NOT EXISTS
        if not Drone_Operator.query.first(): # check if there are records in the DB, if not, insert the first pilot
            db.session.add(Drone_Operator(pilot_name="George Freedom", email="george.freedom@vast_space.com", callsign="Athamantis"))
            db.session.commit()

    # any other actions with the DB

    return app

### Summary

- The old `app/routes.py` file can be deleted = its role has been fully taken over by the route files split into corresponding folders with **Blueprints** (main, api ..).

- Routes are now defined on the Blueprint (`@main_bp.route`), which makes them portable - even to other applications.

- The Application Factory (funkce `app_factory`) becomes a clean "assembly line" that just takes `finished components` and `connects` them to the main application.

### **Practise**

**Connecting a New Module**

Our control system, built on the application factory, is ready for expansion. Our task is to create a new, independent module (Blueprint) for geological drone diagnostics and connect it to our main application.

**Preparation:**
We will use the functional "Assembly Line" application from the previous lesson (Lesson 7) - and extend it.

**Assignment:**

- **"Assembly Line" structure:**
- processor_project/
- wsgi.py
- app/
    - _ _ init _ _.py
    - config.py
    - routes.py
    - **drone_diagnostics/**
        - **_ _ init _ _.py**
        - **routes.py**

1.  **Create a package for the Blueprint:** inside the `app/` folder, create a `drone_diagnostics/` folder.

2.  **Create the Blueprint:** in `app/drone_diagnostics/`, create an `__init__.py` file - here, define a new Blueprint named `'drone'`.

3.  **Prepare routes for the Blueprint:** in `app/drone_diagnostics/`, create a `routes.py` file - here, create at least one route specific to drones - for example, `@drone_bp.route('/ping')`, which returns the text `"Successful connection with drone diagnostics!"`.

4.  **Register the Blueprint:** in the application factory in `app/__init__.py`, import the `drone_bp` blueprint, set its prefix to `url_prefix='/drone'`, and register it on the `app` instance.

5.  **Verify functionality:** run the application and verify that the routes work, and that the new route is available at `http://127.0.0.1:5000/drone/ping`.

### **Project (Homework): Extending the Core with Specialized Modules**

**Mission:** In the last mission, we successfully rebuilt the core of our system into a robust foundation with an application factory. Now it's time for the next step: all our communication protocols and control panels, which are still in a single `routes.py` file, will be divided into specialized, independent modules (Blueprints).


**Target application structure:**
- my_project/
    - wsgi.py
    - app/
        - _ _ init _ _.py
        - database.py
        - models/
            - _ _ init _ _.py
            - crewmember.py
        - data/
            - _ _ init _ _.py
            - asteroid.py
            - crew.py
        - templates/
        - static/
        - config/
        - **main/** Blueprint pro globální / společné routy (podbalíček "main")
            - **_ _ init _ _.py**
            - **routes.py** - routy pro hlavní / společné rozhraní
            - **templates/** - šablony k routám
        - **crew/** Blueprint pro routy s posádkou (podbalíček "crew")
            - **_ _ init _ _.py**
            - **routes.py** - routy pro posádku
            - **forms.py** - formuláře
            - **templates/** - šablony k routám
        - **api/** Blueprint pro API routy (podbalíček "api")
            - **_ _ init _ _.py**
            - **routes.py** - API routy
        - /instance
            - my_database.db

- The file with our **SQLite DB** is in the default **instance** folder next to our application, which is the **root** folder.


**1. Division into Specialized Modules (Blueprints):**
* **Main Bridge (`main`):** Create the `app/main/` **package** for the Blueprint - move all general routes (`/`, `/mission_briefing`, `/target_asteroid`) and their templates here - from the global `/templates` to the `main` blueprint `/templates`, and perform the corresponding imports into `__init__.py`.
* **Crew Management Section (`crew`):** Create the `app/crew/` **package** - move everything related to the crew here: routes (`/crew_members`, `/crew_members/<int:id>`, `/onboarding`), the `NewCrewMemberForm` form, and their templates - from the global `/templates` to the `crew` blueprint `/templates`, and perform the corresponding imports into `__init__.py`.
* **Automated Data Channel (`api`):** Create the `app/api/` package - move all your API endpoints here, and perform the corresponding imports into `__init__.py`.

**2. Reconfiguring Internal Systems (Fixing Imports):**
* By moving files into new packages, their relative paths have changed - we must go through all the files we have moved (`routes.py`, `forms.py`), and **fix the `import` statements** so that they correspond to the new structure.
* *Tip: An import that was previously `from .database import db` (from `app/routes.py`) must now, for example, in `app/crew/routes.py`, be updated to `from ..database import db`, because we need to go one level up.*

**3. Recalibrating Navigation Links (`url_for`):**
* After moving the routes to Blueprints, their internal names (endpoints) have changed (e.g., from `'homepage'` to `'main.homepage'`).
* We need to perform a refactoring = go through all our templates and our code and **update all uses of `url_for()`** so that they use the new, full endpoint names.
* *Tip: For links *within* a single Blueprint, it is possible to use a relative path with a dot (`.`).*

**4. Final Module Integration:**
* In the application factory (`app/__init__.py`), import and **register** all three new Blueprints.

**5. Systems Check and Functionality Verification:**
* Verify that the specific templates in the Blueprints correctly inherit from the shared `base.html` in the main `app/templates/` folder.
* Verify that all links and redirects throughout the application still work correctly, even though they are now split between different Blueprints.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom